In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vibe/4 models/mbti_chunked_clean.csv')

In [4]:
df.head()

,chunk_id,label,text
0,1,intj,the pope is infallible this is a catholic dogm...
1,2,intj,martin said that george floyd was in heaven it...
2,3,intj,while supporting abortion lol abortion again i...
3,4,intj,views on predestination exist in the catholic ...
4,5,intj,a little kitty cat i mean the real dangerousne...


In [5]:
df['label'].unique()

array(['intj', 'intp', 'entj', 'entp', 'infj', 'infp', 'enfj', 'enfp',
       'istj', 'isfj', 'estj', 'esfj', 'istp', 'isfp', 'estp', 'esfp'],
      dtype=object)

In [6]:
# Séparer chaque lettre du type MBTI dans une nouvelle colonne
df['ie'] = df['label'].str[0].str.upper()  # I ou E
df['ns'] = df['label'].str[1].str.upper()  # N ou S
df['tf'] = df['label'].str[2].str.upper()  # T ou F
df['jp'] = df['label'].str[3].str.upper()  # J ou P

In [7]:
df['I'] = (df['ie'] == 'I').astype(int)
df['N'] = (df['ns'] == 'N').astype(int)
df['T'] = (df['tf'] == 'T').astype(int)
df['J'] = (df['jp'] == 'J').astype(int)

In [8]:
df.head()

,chunk_id,label,text,ie,ns,tf,jp,I,N,T,J
0,1,intj,the pope is infallible this is a catholic dogm...,I,N,T,J,1,1,1,1
1,2,intj,martin said that george floyd was in heaven it...,I,N,T,J,1,1,1,1
2,3,intj,while supporting abortion lol abortion again i...,I,N,T,J,1,1,1,1
3,4,intj,views on predestination exist in the catholic ...,I,N,T,J,1,1,1,1
4,5,intj,a little kitty cat i mean the real dangerousne...,I,N,T,J,1,1,1,1


In [9]:
print(f"balance of the target ie : {df['I'].value_counts(normalize=True)}")
print(f"balance of the target ns :{df['N'].value_counts(normalize=True)}")
print(f"balance of the target tf :{df['T'].value_counts(normalize=True)}")
print(f"balance of the target jp :{df['J'].value_counts(normalize=True)}")

balance of the target ie : I
1    0.664045
0    0.335955
Name: proportion, dtype: float64
balance of the target ns :N
1    0.785835
0    0.214165
Name: proportion, dtype: float64
balance of the target tf :T
0    0.582731
1    0.417269
Name: proportion, dtype: float64
balance of the target jp :J
0    0.543701
1    0.456299
Name: proportion, dtype: float64


In [10]:
tokenizer = AutoTokenizer.from_pretrained("nreimers/MiniLM-L6-H384-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [35]:
def preproc(df, y):
  X_train, X_test, y_train, y_test = train_test_split(df['text'], y)
  X_train_tok = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="tf", max_length=512)
  X_test_tok = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="tf", max_length=512)
  y_train_t = tf.constant(y_train, dtype=tf.float32)
  y_test_t = tf.constant(y_test, dtype=tf.float32)
  return X_train_tok, X_test_tok, y_train_t, y_test_t

In [36]:
from transformers import TFAutoModelForSequenceClassification, create_optimizer
from keras.losses import BinaryCrossentropy
import tensorflow as tf

In [37]:
X_train_tok, X_test_tok, y_train_t, y_test_t = preproc(df, df['I'])

In [42]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_tok), y_train_t))
train_dataset = train_dataset.shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_tok), y_test_t))
test_dataset = test_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [44]:
steps_per_epoch = len(X_train_tok) // 64

num_train_steps = steps_per_epoch * 10  # e.g. 10 epochs

optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=0
)

model = TFAutoModelForSequenceClassification.from_pretrained(
    "nreimers/MiniLM-L6-H384-uncased",
    from_pt=True,
    num_labels=1)

model.compile(
    optimizer=optimizer,
    loss=BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"]
    )

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
model.fit(train_dataset, validation_data=test_dataset, epochs=5)

Epoch 1/5
389/389 [==============================] - 135s 297ms/step - loss: nan - accuracy: 0.3347 - val_loss: nan - val_accuracy: 0.3397
Epoch 2/5
389/389 [==============================] - 113s 292ms/step - loss: nan - accuracy: 0.3347 - val_loss: nan - val_accuracy: 0.3397
Epoch 3/5
389/389 [==============================] - 113s 292ms/step - loss: nan - accuracy: 0.3347 - val_loss: nan - val_accuracy: 0.3397
Epoch 4/5
389/389 [==============================] - 113s 292ms/step - loss: nan - accuracy: 0.3347 - val_loss: nan - val_accuracy: 0.3397
Epoch 5/5
389/389 [==============================] - 113s 292ms/step - loss: nan - accuracy: 0.3347 - val_loss: nan - val_accuracy: 0.3397
